In [ ]:
# Install required packages
!pip install google-api-python-client gtts moviepy gradio transformers reportlab


In [ ]:
import gradio as gr
import os
import time
import textwrap
from googleapiclient.discovery import build
from gtts import gTTS
from moviepy.editor import ImageSequenceClip, AudioFileClip
from PIL import Image
import requests
from io import BytesIO
from transformers import AutoTokenizer, AutoModelForCausalLM
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

# API Keys (Replace with your own keys)
API_KEY = 'API_KEY'
SEARCH_ENGINE_ID = 'SEARCH_ENGINE_ID'  # Google Custom Search Engine ID


In [ ]:
# Initialize the text generation model
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')
text_model = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-1.3B')

def generate_text(prompt, max_length=500):
    start_time = time.time()
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = text_model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        do_sample=True
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    elapsed_time = time.time() - start_time
    return generated_text, elapsed_time


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [ ]:
def get_text_results(prompt, num_results=3):
    start_time = time.time()
    service = build('customsearch', 'v1', developerKey=API_KEY)
    request = service.cse().list(
        q=prompt,
        cx=SEARCH_ENGINE_ID,
        num=num_results,
        fields='items(title,snippet,link)'
    )
    response = request.execute()

    results = []
    for item in response.get('items', []):
        title = item.get('title')
        snippet = item.get('snippet')
        link = item.get('link')
        results.append(f"**{title}**\n{snippet}\nLink: {link}\n")

    elapsed_time = time.time() - start_time
    return "\n\n".join(results), elapsed_time


In [ ]:
def get_images(query, num_images=5):
    start_time = time.time()
    service = build('customsearch', 'v1', developerKey=API_KEY)
    request = service.cse().list(
        q=query,
        cx=SEARCH_ENGINE_ID,
        searchType='image',
        num=num_images,
        imgSize='MEDIUM',
        fields='items(link)'
    )
    response = request.execute()
    image_links = [item['link'] for item in response.get('items', [])]
    elapsed_time = time.time() - start_time
    return image_links, elapsed_time

def download_images(image_links, output_dir='images'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_paths = []
    for idx, img_url in enumerate(image_links):
        try:
            response = requests.get(img_url)
            img = Image.open(BytesIO(response.content))
            image_path = os.path.join(output_dir, f'image_{idx + 1}.png')
            img.save(image_path)
            image_paths.append(image_path)
        except Exception as e:
            print(f'Failed to download image {idx + 1}: {e}')
    return image_paths


In [ ]:
def create_pdf(text, images, filename='output.pdf'):
    start_time = time.time()
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Add text
    wrapped_text = textwrap.fill(text, 80)
    text_object = c.beginText(40, height - 40)
    text_object.setFont("Helvetica", 12)
    for line in wrapped_text.split('\n'):
        text_object.textLine(line)
    c.drawText(text_object)

    # Add images
    y_position = height - 400
    for img_path in images:
        c.drawImage(img_path, 100, y_position, width=400, height=200)
        y_position -= 220  # Adjust for next image

    c.save()
    elapsed_time = time.time() - start_time
    return filename, elapsed_time


In [ ]:
def generate_content(prompt):
    # Generate AI text
    generated_text, text_time = generate_text(prompt)

    # Fetch web results
    web_results, web_time = get_text_results(prompt)

    # Retrieve images
    image_links, image_time = get_images(prompt)
    image_paths = download_images(image_links)

    # Create PDF
    pdf_filename, pdf_time = create_pdf(generated_text, image_paths)

    # Performance metrics
    metrics = (
        f"Text Generation Time: {text_time:.2f} seconds\n"
        f"Web Search Time: {web_time:.2f} seconds\n"
        f"Image Retrieval Time: {image_time:.2f} seconds\n"
        f"PDF Creation Time: {pdf_time:.2f} seconds\n"
    )

    return generated_text, web_results, image_paths, pdf_filename, metrics

# Gradio Interface setup
interface = gr.Interface(
    fn=generate_content,
    inputs=gr.Textbox(label="Enter a prompt"),
    outputs=[
        gr.Textbox(label="Generated Text"),
        gr.Textbox(label="Relevant Web Results"),
        gr.Gallery(label="Relevant Images"),
        gr.File(label="Download Generated PDF"),
        gr.Textbox(label="Performance Metrics")
    ]
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://319707357d8cab656d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
